In [8]:
from ox_db.ai.vector import Model
vec = Model()

/home/lokesh/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/lokesh/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
data = ["lok","name","time","date","lokesh","uranium"]
query = "lokesh"
encoaded_vec_a = vec.encode("lokesh")
encoaded_vec_b = vec.encode("lokesh")
encoaded_data = []
for i in range(len(data)):
    encoaded_data.append(vec.encode(data[i]))


In [10]:
sformat = "ed"

In [11]:
vec.search(query,doc_embed=encoaded_data,by=sformat)

{'idx': [4, 0, 1, 5, 2, 3],
 'sim_score': [0.0,
  0.5131477314849803,
  1.1526391002573002,
  1.2041050630355528,
  1.2183306817525545,
  1.2569165109747262]}

In [12]:
vec.sim(encoaded_vec_a,encoaded_vec_b,sformat)

0.0

In [13]:
from ox_db.db.types import PushModel

PushModel(data="hib c uj jc dei ic").model_dump()

{'data': 'hib c uj jc dei ic',
 'embeddings': True,
 'description': None,
 'metadata': None,
 'uid': None}

In [3]:
from ox_db.db.log import Oxdb

db = Oxdb()

/home/lokesh/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/lokesh/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
push = {"data":"testing for shell implimentation"}
search = {"query":"test shell"}

In [7]:
db.doc.push(**push)
db.doc.search(**search)

{'entries': 1,
 'hid': ['f984a05201dba017b84ea76371e4449aa1546cc633992e0af8a8b878f3e769eb'],
 'data': ['testing for shell implimentation'],
 'description': [None],
 'sim_score': [0.7160544124671222],
 'index': [{'hid': 'f984a05201dba017b84ea76371e4449aa1546cc633992e0af8a8b878f3e769eb',
   'uid': 'uid',
   'doc': 'log-[18_08_2024]',
   'time': '02:17:34',
   'date': '18-08-2024'}],
 'embeddings': []}

In [100]:
class OxdbShell:
    def __init__(self, oxdb):
        self.oxdb = oxdb
        self.command_mappings = {
            "oxdb.info": "oxdb.info()",
            "info": "oxdb.info()",
            "oxdb.doc.info": "oxdb.doc.info()",
            "doc.info": "oxdb.doc.info()",
            "doc info": "oxdb.doc.info()",
            'oxdb.get("': "oxdb.get_db({})",
            'oxdb get "': "oxdb.get_db({})",
            'oxdb get("': "oxdb.get_db({})",
            'get("': "oxdb.get_db({})",
            'get "': "oxdb.get_db({})",
            'oxdb.doc.get("': "oxdb.doc.get_doc({})",
            'oxdb doc get "': "oxdb.doc.get_doc({})",
            'oxdb doc get("': "oxdb.doc.get_doc({})",
            'doc.get("': "oxdb.doc.get_doc({})",
            'doc get "': "oxdb.doc.get_doc({})",
            "oxdb.doc.push(": "oxdb.doc.push({})",
            "doc.push(": "oxdb.doc.push({})",
            "doc push(": "oxdb.doc.push({})",
            "push(": "oxdb.doc.push({})",
            "oxdb.doc.pull(": "oxdb.doc.pull({})",
            "doc.pull(": "oxdb.doc.pull({})",
            "doc pull(": "oxdb.doc.pull({})",
            "pull(": "oxdb.doc.pull({})",
            "oxdb.doc.search(": "oxdb.doc.search({})",
            "doc.search(": "oxdb.doc.search({})",
            "doc search(": "oxdb.doc.search({})",
            "search(": "oxdb.doc.search({})",
        }

    def validate_command(self, shell_command: str):
        for key in self.command_mappings.keys():
            if shell_command.startswith(key):
                return True
        return False

    def translate_command(self, shell_command: str):
        for key, value in self.command_mappings.items():
            if shell_command.startswith(key):
                translated_command = value
                
                if "{}" not in value:
                    return value
                # Handle arguments in quotes for methods like get, push, pull, and search
                if not "(" in shell_command and '"' in shell_command and '"' in shell_command and "{}" in value:
                    arg_start = shell_command.find('"')
                    arg_end = shell_command.rfind('"') + 1
                    if arg_start != -1 and arg_end != -1:
                        args = shell_command[arg_start:arg_end]
                        translated_command = value.replace("{}", args)

                    print(1, translated_command)
                    return translated_command
                
                # Handle dictionary-style arguments for push, pull, search
                if "{}" not in shell_command and "(" in shell_command and ")" in shell_command and "{}" in value:
                    args = shell_command[
                        shell_command.find("(") + 1 : shell_command.rfind(")")
                    ]

                    args_string = f"{args}"

                    translated_command = value.replace("{}", args_string)

                    print(2, translated_command)

                    return translated_command
                                # Handle dictionary-style arguments for push, pull, search
                if "{}" in shell_command and "(" in shell_command and ")" in shell_command and "{}" in value:
                    args = shell_command[
                        shell_command.find("(") + 1 : shell_command.rfind(")")
                    ]
                    args_string = args
                    if "**" != args[:2]:
                        args_string = "**{"+args+"}"
                    

                    translated_command = value.replace("{}", args_string)

                    print(2, translated_command)

                    return translated_command

        return shell_command


#

In [101]:
oxdb_shell = OxdbShell('')
shell_command = 'oxdb get "mydb"'
if oxdb_shell.validate_command(shell_command):
    translated = oxdb_shell.translate_command(shell_command)
    print(translated)
    # This will output: oxdb.doc.get_doc("docname")


1 oxdb.get_db("mydb")
oxdb.get_db("mydb")


In [104]:

def test_oxdb_shell():
    oxdb_shell = OxdbShell(oxdb=None)  # Mock `oxdb` as None for testing purposes
    
    # Test cases for `oxdb.info()` command
    assert oxdb_shell.validate_command('oxdb.info')
    assert oxdb_shell.translate_command('oxdb.info') == 'oxdb.info()'
    
    assert oxdb_shell.validate_command('info')
    assert oxdb_shell.translate_command('info') == 'oxdb.info()'
    
    # Test cases for `oxdb.doc.info()` command
    assert oxdb_shell.validate_command('oxdb.doc.info')
    assert oxdb_shell.translate_command('oxdb.doc.info') == 'oxdb.doc.info()'
    
    assert oxdb_shell.validate_command('doc.info')
    assert oxdb_shell.translate_command('doc.info') == 'oxdb.doc.info()'
    
    assert oxdb_shell.validate_command('doc info')
    assert oxdb_shell.translate_command('doc info') == 'oxdb.doc.info()'
    
    # Test cases for `oxdb.get_db("dbname")` command
    assert oxdb_shell.validate_command('oxdb.get("mydb")')
    assert oxdb_shell.translate_command('oxdb.get("mydb")') == 'oxdb.get_db("mydb")'
    
    assert oxdb_shell.validate_command('get("mydb")')
    assert oxdb_shell.translate_command('get("mydb")') == 'oxdb.get_db("mydb")'
    
    assert oxdb_shell.validate_command('oxdb get "mydb"')
    assert oxdb_shell.translate_command('oxdb get "mydb"') == 'oxdb.get_db("mydb")'
    
    assert oxdb_shell.validate_command('get "mydb"')
    assert oxdb_shell.translate_command('get "mydb"') == 'oxdb.get_db("mydb")'
    
    # Test cases for `oxdb.get_db("dbname","path")` command
    assert oxdb_shell.validate_command('oxdb.get("mydb","path")')
    assert oxdb_shell.translate_command('oxdb.get("mydb","path")') == 'oxdb.get_db("mydb","path")'
    
    assert oxdb_shell.validate_command('get("mydb","path")')
    assert oxdb_shell.translate_command('get("mydb","path")') == 'oxdb.get_db("mydb","path")'
    
    assert oxdb_shell.validate_command('oxdb get("mydb","path")')
    assert oxdb_shell.translate_command('oxdb get("mydb","path")') == 'oxdb.get_db("mydb","path")'
    
    assert oxdb_shell.validate_command('get("mydb","path")')
    assert oxdb_shell.translate_command('get("mydb","path")') == 'oxdb.get_db("mydb","path")'
    
    # Test cases for `oxdb.doc.get_doc("docname")` command
    assert oxdb_shell.validate_command('oxdb.doc.get("mydoc")')
    assert oxdb_shell.translate_command('oxdb.doc.get("mydoc")') == 'oxdb.doc.get_doc("mydoc")'
    
    assert oxdb_shell.validate_command('doc.get("mydoc")')
    assert oxdb_shell.translate_command('doc.get("mydoc")') == 'oxdb.doc.get_doc("mydoc")'
    
    assert oxdb_shell.validate_command('doc get "mydoc"')
    assert oxdb_shell.translate_command('doc get "mydoc"') == 'oxdb.doc.get_doc("mydoc")'
    
    # Test cases for `oxdb.doc.push(**data)` command
    data = '**{"uid": "177473", "document": "this is test document"}'
    assert oxdb_shell.validate_command(f'oxdb.doc.push({data})')
    assert oxdb_shell.translate_command(f'oxdb.doc.push({data})') == f'oxdb.doc.push({data})'
    
    assert oxdb_shell.validate_command(f'doc.push({data})')
    assert oxdb_shell.translate_command(f'doc.push({data})') == f'oxdb.doc.push({data})'
    
    assert oxdb_shell.validate_command(f'doc push({data})')
    assert oxdb_shell.translate_command(f'doc push({data})') == f'oxdb.doc.push({data})'
    
    assert oxdb_shell.validate_command(f'push({data})')
    assert oxdb_shell.translate_command(f'push({data})') == f'oxdb.doc.push({data})'
    
    # Test cases for `oxdb.doc.pull(**data)` command
    data = '**{"uid": "177473"}'
    assert oxdb_shell.validate_command(f'oxdb.doc.pull({data})')
    assert oxdb_shell.translate_command(f'oxdb.doc.pull({data})') == f'oxdb.doc.pull({data})'
    
    assert oxdb_shell.validate_command(f'doc.pull({data})')
    assert oxdb_shell.translate_command(f'doc.pull({data})') == f'oxdb.doc.pull({data})'
    
    assert oxdb_shell.validate_command(f'doc pull({data})')
    assert oxdb_shell.translate_command(f'doc pull({data})') == f'oxdb.doc.pull({data})'
    
    assert oxdb_shell.validate_command(f'pull({data})')
    assert oxdb_shell.translate_command(f'pull({data})') == f'oxdb.doc.pull({data})'
    
    # Test cases for `oxdb.doc.search(**data)` command
    data = '{"query": "test"}'
    assert oxdb_shell.validate_command(f'oxdb.doc.search({data})')
    assert not oxdb_shell.translate_command(f'oxdb.doc.search({data})') == f'oxdb.doc.search(**{data})'
    
    assert oxdb_shell.validate_command(f'doc.search({data})')
    assert not oxdb_shell.translate_command(f'doc.search({data})') == f'oxdb.doc.search(**{data})'
    
    assert oxdb_shell.validate_command(f'doc search({data})')
    assert not oxdb_shell.translate_command(f'doc search({data})') == f'oxdb.doc.search(**{data})'
    
    assert oxdb_shell.validate_command(f'search({data})')
    assert not oxdb_shell.translate_command(f'search({data})') == f'oxdb.doc.search(**{data})'

    print("All tests passed!")

# Run the test function
test_oxdb_shell()


2 oxdb.get_db("mydb")
2 oxdb.get_db("mydb")
1 oxdb.get_db("mydb")
1 oxdb.get_db("mydb")
2 oxdb.get_db("mydb","path")
2 oxdb.get_db("mydb","path")
2 oxdb.get_db("mydb","path")
2 oxdb.get_db("mydb","path")
2 oxdb.doc.get_doc("mydoc")
2 oxdb.doc.get_doc("mydoc")
1 oxdb.doc.get_doc("mydoc")
2 oxdb.doc.push(**{"uid": "177473", "document": "this is test document"})
2 oxdb.doc.push(**{"uid": "177473", "document": "this is test document"})
2 oxdb.doc.push(**{"uid": "177473", "document": "this is test document"})
2 oxdb.doc.push(**{"uid": "177473", "document": "this is test document"})
2 oxdb.doc.pull(**{"uid": "177473"})
2 oxdb.doc.pull(**{"uid": "177473"})
2 oxdb.doc.pull(**{"uid": "177473"})
2 oxdb.doc.pull(**{"uid": "177473"})
2 oxdb.doc.search({"query": "test"})
2 oxdb.doc.search({"query": "test"})
2 oxdb.doc.search({"query": "test"})
2 oxdb.doc.search({"query": "test"})
All tests passed!
